In [13]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from docx import Document
import pandas as pd
import urllib.parse

# 주식 종목명을 사용하여 종목 코드 검색
def search_stock_code(stock_name):
    encoded_name = urllib.parse.quote(stock_name)
    url = f'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&orderMode=3&orderStat=D&searchName={encoded_name}'
    df = pd.read_html(url, header=0, encoding='euc-kr')[0]
    stock_code = df.iloc[0]['종목코드']
    return str(stock_code).zfill(6)

# 네이버 금융 페이지에서 주식 정보 스크래핑
def scrape_stock_data(stock_code, start_date, end_date):
    url = f'https://finance.naver.com/item/sise_day.nhn?code={stock_code}&page=1'

    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    price_data = get_price_data(soup)
    plot_stock_data(price_data)

# 주가 데이터를 그래프로 그리는 함수
def plot_stock_data(price_data):
    dates = [data[0] for data in price_data]
    prices = [data[1] for data in price_data]

    plt.figure(figsize=(12, 6))
    plt.plot(dates, prices, marker='o', linestyle='-', color='b')
    plt.title('주가 변동 그래프')
    plt.xlabel('날짜')
    plt.ylabel('주가')
    plt.grid()
    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.show()

# 네이버 금융에서 주식 가격 데이터 가져오기
def get_price_data(soup):
    price_data = []

    # 웹 페이지에서 데이터를 추출
    table = soup.find('table')
    if table:
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 7:  # 데이터가 있는 행만 처리
                date = cols[0].get_text()
                price = int(cols[1].get_text().replace(',', ''))  # 쉼표(,) 제거 후 정수로 변환
                price_data.append((date, price))

    return price_data

# 매출 현황 보고서 생성 함수
def create_sales_report(report_data, stock_name):
    doc = Document()
    doc.add_heading(f'{stock_name} 매출 현황 보고서', 0)

    for month, sales in report_data.items():
        doc.add_heading(f'{month} 매출', level=1)
        doc.add_paragraph(f'매출: {sales} 원')

    doc.save(f'{stock_name}_sales_report.docx')
    print(f'{stock_name} 매출 현황 보고서가 생성되었습니다.')

if __name__ == "__main__":
    stock_name = input("주식 종목명을 입력하세요: ")
    stock_code = search_stock_code(stock_name)
    print(f'{stock_name}의 종목 코드는 {stock_code} 입니다.')

    start_date = input("시작 날짜(yyyy-mm-dd)를 입력하세요: ")
    end_date = input("종료 날짌(yyyy-mm-dd)를 입력하세요: ")

    # 스크래핑 및 그래프 생성
    scrape_stock_data(stock_code, start_date, end_date)

    # 매출 데이터를 딕셔너리 형태로 준비
    report_data = {
        '2023-01': 1000000,
        '2023-02': 1200000,
        # 추가 데이터 입력
    }

    # 매출 보고서 생성
    create_sales_report(report_data, stock_name)

    # 현대차(Hyundai Motor) 주가 정보
    hyundai_code = search_stock_code('현대차')
    hyundai_start_date = '2023-01-01'
    hyundai_end_date = '2023-12-31'
    scrape_stock_data(hyundai_code, hyundai_start_date, hyundai_end_date)

    # 삼성전자(Samsung Electronics) 주가 정보
    samsung_code = search_stock_code('삼성전자')
    samsung_start_date = '2023-01-01'
    samsung_end_date = '2023-12-31'
    scrape_stock_data(samsung_code, samsung_start_date, samsung_end_date)


FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?